#백준 1753 번: 최단경로

##문제: 방향그래프가 주어지면 주어진 시작점에서 다른 모든 정점으로의 최단 경로를 구하는 프로그램을 작성하시오. 단, 모든 간선의 가중치는 10 이하의 자연수이다.

##입력:
첫째 줄에 정점의 개수 V와 간선의 개수 E가 주어진다. (1 ≤ V ≤ 20,000, 1 ≤ E ≤ 300,000) 모든 정점에는 1부터 V까지 번호가 매겨져 있다고 가정한다. 둘째 줄에는 시작 정점의 번호 K(1 ≤ K ≤ V)가 주어진다. 셋째 줄부터 E개의 줄에 걸쳐 각 간선을 나타내는 세 개의 정수 (u, v, w)가 순서대로 주어진다. 이는 u에서 v로 가는 가중치 w인 간선이 존재한다는 뜻이다. u와 v는 서로 다르며 w는 10 이하의 자연수이다. 서로 다른 두 정점 사이에 여러 개의 간선이 존재할 수도 있음에 유의한다.


##출력:
첫째 줄부터 V개의 줄에 걸쳐, i번째 줄에 i번 정점으로의 최단 경로의 경로값을 출력한다. 시작점 자신은 0으로 출력하고, 경로가 존재하지 않는 경우에는 INF를 출력하면 된다.

 <img src="img/1753/1753_1.png" width="800" height="300">


#Priority Queue(우선순위 큐)

먼저 들어간 선입선출(FIFO)식 자료구조인 Queue와는 다르게, Priority Queue는 데이터가 제거될 때 데이터들 간의 우선순위를 선정하여 우선순위가 높은 데이터를 먼저 제거하는 방식이다.


즉,내부적으로 데이터를 정렬하여 우선순위를 어떻게 둘 것인지에 따라 가장 작은 값 또는 가장 큰 값 등을 먼저 제거하는 형태라고 생각할 수 있다. PriorityQueue에서 값을 추가하거나 뺄 때(put/get)는 O(log n)의 시간 복잡도를 가진다. 상황에 따라 다를 수 있으나 보통 리스트를 정렬할 때 O(n log n)이 소요된다고 하므로 정렬하여 순서대로 뽑아오고 싶으면 이 PriorityQueue를 사용해보자.


기본적인 사용법은 queue라는 파이썬 내장 모듈에 있는 PriorityQueue라는 클래스를 사용하면 된다.

In [ ]:
###PriorityQueue
from queue import PriorityQueue

##우선순위 큐 생성
P_que = PriorityQueue() #만약 특정 최대 크기를 가진 우선순위 큐를 생성 시 maxsize 라는 매개변수를 사용하여 생성

##우선순위 큐에 원소 추가(put 메서드)
P_que.put(10)
P_que.put(5)
P_que.put(7)
P_que.put(1)
P_que.put(33)

print(P_que)
print(f'P_que: {(P_que.queue)}')  #값을 확인할 시 객체.queue속성으로 접근하여 확인, 멀티스레드 사용시에는 mutex라는 잠금을 사용한 뒤 읽어야함

print('---------------------------------')

##우선순위 큐에서 원소 제거(get 메서드,default로 작은 값부터 제거됨)
print(P_que.get())
print(P_que.get())
print(P_que.get())
print(P_que.get())
print(P_que.get())

print('---------------------------------')

##정렬 기준 변경(다른 기준으로 원소가 정렬되기를 원한다면 (우선순위,값)의 튜플 형태로 데이터 추가&제거)
P_que.put((3,10))
P_que.put((1,5))
P_que.put((2,7))
P_que.put((5,1))
P_que.put((4,33))

print(P_que.get()[1])
print(P_que.get()[1])
print(P_que.get()[1])
print(P_que.get()[1])
print(P_que.get()[1])

print('---------------------------------')

##내림차순으로 반환(내림차순은 보통 값에 음수를 씌워 우선순위로 전달)
P_que.put((-10,10))
P_que.put((-5,5))
P_que.put((-7,7))
P_que.put((-1,1))
P_que.put((-33,33))

print(P_que.get()[1])
print(P_que.get()[1])
print(P_que.get()[1])
print(P_que.get()[1])
print(P_que.get()[1])



P_que: [1, 5, 7, 10, 33]
---------------------------------
1
5
7
10
33
---------------------------------
5
7
10
33
1
---------------------------------
33
10
7
5
1


#Dijkstra Algorithm(다익스트라 알고리즘)

다익스트라 알고리즘은 그래프의 한 노드(정점)에서 다른 모든 노드까지의 최단거리를 각각 구하는 알고리즘으로, 에지(간선)가 모두 양수일 때 사용하며, 가중/방향 유무에 상관없이 적용 가능하다는 장점이 있다. 시간복잡도는 O(E log V)이며, 여기서 V는 노드 개수, E는 에지 개수를 나타낸다.


구현방법은 '단계마다 방문하지 않은 노드 중에서 가장 최단거리가 짧은 노드를 선택한 뒤, 그 노드를 거쳐 가는 경우를 확인하여 최단거리를 갱신' 하는 것인데 구체적인 내용은 아래와 같다.


##1. 그래프 인접 리스트화:

인접 리스트먼저 주어진 그래프를 노드와 인접 리스트로써 나타낸다. 여기서 기준 노드에 인접하는 노드들을 (노드,가중치) 형태로 나타내어 준다. 또한 방향 그래프일 시 기준 노드에서 나오는 방향일 때 인접노드들(엣지)을 표현해주면 된다. 예를 들어 1번 노드에 가중치 3인 방향엣지로 4번노드가 연결되어 있다면 1-[4,3] 형태로 표현해주는 것이다.

##2. 최단 거리 리스트 초기화:

시작 노드로부터의 각 노드까지의 최단거리값을 담을 리스트를 만들고 값을 초기화한다. 출발노드는 0, 나머지노드들은 ∞를 담아주면 된다. 이 ∞은 실제 구현에서 주어진 에지의 합보다 충분히 큰 수로 정하면 된다.

##3. 값이 가장 작은 노드를 고름:

맨 처음에는 시작노드인 0값이 가장 작으니 시작노드에서 출발하게 된다.

##4. 최단 거리 리스트 업데이트:

3에서 선택한 노드를 기준으로 가중치를 적용하여 연결된 노드들의 최단 거리 리스트를 업데이트 한다. 인접 리스트를 이용하여 인접한 노드들을 확인하며, 3에서 선택한 노드의 최단거리 값에 가중치를 더한 값을 인접노드의 최단거리 값과 비교하여 더 작은 값으로 업데이트 해준다. 또한 한번 방문한 노드(3에서 선택한 노드)는 다시 방문하지 않도록 방문처리 해준다.

##5. 과정 3-4를 반복하여 최단거리 리스트 완성:

모든 노드를 방문할 때까지 과정3과4를 반복하여 최단거리 리스트를 완성한다.


In [ ]:
###백준 1753 문제-다익스트라 알고리즘
import sys
from queue import PriorityQueue

##입력 받기
V,E = map(int,input().split())  #V는 노드 개수, E는 엣지 개수
S = int(input())  #S는 시작 노드

##그래프 인접 리스트화
#여기서 V+1개 생성해주는 이유는 인덱스 번호와 노드번호를 맞추기 위해서이다. 이러면 0번 노드가 추가로 생기는 셈이지만 활용하지 않을 것으로, 1~V번 노드만 사용할 것이다.
graph = [[] for _ in range(V+1)]
#생성한 V개의 리스트에 엣지들을 표현한 튜플들을 넣어준다. 여기서 각 리스트는 기준 노드를 의미하기에 u번째 리스트를 graph[u]로 접근하면 u번 노드와 인접한 노드들과 가중치를 확인할 수 있다.
for _ in range(E):
  u,v,w = map(int,input().split())  #u는 시작 노드, v는 도착 노드, w는 가중치 -> 엣지들을 표현
  graph[u].append((v,w))

5 6
1
5 1 1
1 2 2
1 3 3
2 3 4
2 4 5
3 4 6


In [ ]:
print(graph)  #저장한 그래프 확인

[[], [(2, 2), (3, 3)], [(3, 4), (4, 5)], [(4, 6)], [], [(1, 1)]]


In [ ]:
##최단거리 리스트와 방문목록 리스트 생성 및 값 초기화
dist = [sys.maxsize] * (V+1)  #sys.maxsize로 무한에 해당하는 수를 넣어주고, 역시 노드 번호와 일치시키기 위해 V+1개 생성
visited = set()  #방문한 노드번호를 적어줄 집합(검색 효율성을 위해 리스트 대신 집합을 사용)

dist[S] = 0 #S를 시작점으로 설정

In [ ]:
print(dist)  #sys.maxsize는 int자료형 중 사용할 수 있는 가장 큰 수이다.

[9223372036854775807, 0, 9223372036854775807, 9223372036854775807, 9223372036854775807, 9223372036854775807]


In [ ]:
##우선순위 큐 생성, 큐에 시작점 넣어주기
q = PriorityQueue() #우선순위 큐에는 (우선순위,값) = (인접 노드의 최단거리 값 , 인접노드) 형태로 추가해줄 것임.
q.put((0,S))  #맨 처음에는 우선순위 0 과 함께 시작노드 번호 S를 넣어줌.

In [ ]:
##최단거리 리스트 반복 업데이트
while q.qsize() > 0:
  node = q.get()[1] #우선순위에 따라 뽑아낸 튜플에서 인덱스1에 해당하는 값 -> 기준노드(번호)

  #만약 그 노드를 방문했다면 skip
  if node in visited:
    continue

  visited.add(node) #방문하지 않았다면 방문 집합에 노드번호를 추가하고 진행
  print(f'visited = {visited} , node = {node}')

  for adj in graph[node]:
    print(f'adj = {adj}') #adj는 인접노드로써 (노드번호,가중치)형태의 튜플이 담겨있음
    next = adj[0] #인접한 다음노드번호
    weight = adj[1] #가중치
    if dist[next] > dist[node] + weight:  #만약 다음 노드의 최단거리가 기준 노드의 최단거리에 가중치를 더한 값보다 크면
      dist[next] = dist[node] + weight  #그 새로운 작은 값으로써 업데이트
      print(f'dist = {dist}')
      q.put((dist[next],next))  #인접 노드번호를 가중치를 우선순위로 삼아 q에 추가
      print(f'q = {q.queue}')

  print('------------------------')

visited = {1} , node = 1
adj = (2, 2)
dist = [9223372036854775807, 0, 2, 9223372036854775807, 9223372036854775807, 9223372036854775807]
q = [(2, 2)]
adj = (3, 3)
dist = [9223372036854775807, 0, 2, 3, 9223372036854775807, 9223372036854775807]
q = [(2, 2), (3, 3)]
------------------------
visited = {1, 2} , node = 2
adj = (3, 4)
adj = (4, 5)
dist = [9223372036854775807, 0, 2, 3, 7, 9223372036854775807]
q = [(3, 3), (7, 4)]
------------------------
visited = {1, 2, 3} , node = 3
adj = (4, 6)
------------------------
visited = {1, 2, 3, 4} , node = 4
------------------------


위 결과를 실행 결과를 보면 ∞에 해당하는 값이 마지막 dist에 담겨있는 것을 확인 할 수 있는데, 이는 마지막까지 방문될 수 없는 노드로써, 시작노드로부터 갈 수 없는 노드라는 뜻이다.

또한 왜 다음 기준노드를 선택하는 과정에 있어서, 우선순위 큐에서 값을 뽑아와 가중치가 낮은 인접노드를 다음 기준노드로 선정할까? 이는 탐색 순서를 최단거리 계산에 맞추어 최적화하려는 의도에서이다.

만약 BFS처럼 Queue로 저 부분을 구현한다고 해도 최단거리 리스트는 최종적으로 모두 업데이트 되겠지만, 시간복잡도가 O(E log V) -> O(VE)로 확 높아진다. 이는 Queue로 뽑는다면 노드를 선정할 때 dist값과 무관한 순서로 뽑아오게 되므로 다음 노드가 최단거리라는 것을 알지 못한 채 선정되는 것이고, 이 때문에 중복방문이 될 수 있는 것 때문이다.

우선순위 큐로 구현하는 경우(정석적인 다익스트라) 매 단계마다 dist 값이 가장 작은 노드를 다음 노드로 선정함으로써, 선정 시 그 노드의 최단거리가 확정되며, 이후로는 해당 노드의 거리 갱신이 불필요해지고, 중복 방문을 거의 할 필요가 없어진다.

예를 들어,

A -- (1) --> B -- (1) --> C

A -- (10) --> C

인 경우를 생각해보자.

###우선순위 Queue의 경우

1. A(0) 꺼냄 → B(1), C(10) 갱신

2. B(1) 꺼냄 → C(2) 갱신 (10 → 2)

3. C(2) 꺼냄 → 끝(C는 단 1번 방문)

###일반 Queue의 경우

1. A(0) 꺼냄 → B(1), C(10) 갱신

2. B(1) 꺼냄 → C(2) 갱신 → C를 큐에 다시 넣음

3. C(10) 꺼냄 → 이미 더 짧은 거리(2)가 있으니 패스

4. C(2) 꺼냄 → 끝(C는 2번 방문)


이와 같이 현재까지 가장 짧은 거리인 노드부터 처리하여 중복방문을 피하고자 우선순위 큐를 사용하여 구현하면 된다.

In [ ]:
###최종 출력
for i in range(1, V+1):
  if i in visited:
    print(dist[i])  #노드가 방문 집합에 등록되었다면 최단거리 출력
  else:
    print('INF')  #방문할 수 없는 노드의 경우(dist가 무한으로 되어있는 노드들) INF를 출력

0
2
3
7
INF


In [ ]:
###전체 코드
import sys
input = sys.stdin.readline
from queue import PriorityQueue

V,E = map(int,input().split())
S = int(input())

graph = [[] for _ in range(V+1)]

for _ in range(E):
  u,v,w = map(int,input().split())
  graph[u].append((v,w))

dist = [sys.maxsize] * (V+1)
visited = set()

dist[S] = 0
q = PriorityQueue()
q.put((0,S))

while q.qsize() > 0:
  node = q.get()[1]
  if node in visited:
    continue

  visited.add(node)

  for adj in graph[node]:
    next = adj[0]
    weight = adj[1]
    if dist[next] > dist[node] + weight:
      dist[next] = dist[node] + weight
      q.put((dist[next],next))

for i in range(1, V+1):
  if i in visited:
    print(dist[i])
  else:
    print('INF')

참고사항)
input = sys.stdin.readline

위 코드는 알고리즘 문제풀이에서 입력 성능을 최적화하기 위해 사용하는 코드이다. 단순 input()으로 읽어드린 문자열에서는 줄바꿈 문자(\n)를 제거하고 출력해주는 과정에서 속도가 저하되는데, sys.stdin.readline을 사용하면 개행문자를 포함하여 읽어드리기 때문에 처리속도가 올라간다고 한다. 반면 후자의 경우 프롬프트 메시지(사용자에게 안내해주는 문자)를 사용할 수 없다는 단점이 있다.